In [2]:
import pandas as pd
from prompt import *
# from

In [15]:
# 获取类别
train_df_org = pd.read_excel('../data/data-0520/train.xlsx')
train_df_cot = pd.read_excel('../data/data4cot/cot4train.xlsx')
train_df = pd.concat([train_df_org, train_df_cot], axis=1)

### 1. Prompt

A. Classify

In [16]:
FUND_INQUIRY = 1
FUND_SELECTION = 2
STOCK_INQUIRY = 4
STOCK_SELECTION = 8
prompt_map = {1:fundQuery_cot_prompt, 2:fundSelect_cot_prompt, 4:stockQuery_cot_prompt, 8:stockSelect_cot_prompt, 0:calculation_cot_prompt}


def get_type_from_json(label:str):
    try:
        label_d = eval(label)
    except:
        print(f'error in parsing {label}')
        return None
    api_list = label_d['relevant APIs']
    type_ = 0
    for api in api_list:
        if 'tool_name' not in api:
            print(f'error in parsing {api}')
            continue
        if api['tool_name'] == '基金查询' and api['api_name'] != '查询代码':    # 个别仅仅查询了代码，不算查询（无效）
            type_ |= FUND_INQUIRY
        elif api['tool_name'] == '条件选基':
            type_ |= FUND_SELECTION
        elif api['tool_name'] == '股票查询' and api['api_name'] != '查询代码':
            type_ |= STOCK_INQUIRY
        elif api['tool_name'] == '条件选股':
            type_ |= STOCK_SELECTION
    
    return type_

train_df['type_'] = train_df['label'].apply(get_type_from_json)

B. Product recall

In [25]:
import difflib
from tqdm import tqdm

train_df['query_with_products'] = train_df['query']
train_df['products'] = None
stock_names = pd.read_excel('../data/data-0520/标准名.xlsx',sheet_name='股票标准名')['标准股票名称'].to_list()
fund_names = pd.read_excel('../data/data-0520/标准名.xlsx',sheet_name='基金标准名')['标准基金名称'].to_list()
for index,row in tqdm(train_df.iterrows(),total=train_df.shape[0]):
    if row['type_'] & STOCK_INQUIRY:
        products = difflib.get_close_matches(row['query'],stock_names,n=50,cutoff=0.0001)
        train_df.at[index,'products'] = products
        train_df.at[index,'query_with_products'] += '\n    query中提到的产品标准名可能是：' + '、'.join(products)
        # print(f'index: {index}, query: {row["query"]}\nproducts: {products}\nnew_query: {train_df.at[index,"query_with_products"]}')
    if row['type_'] & FUND_INQUIRY:
        products = difflib.get_close_matches(row['query'],fund_names,n=50,cutoff=0.0001)
        train_df.at[index,'products'] = products
        train_df.at[index,'query_with_products'] += '\n    query中提到的产品标准名可能是：' + '、'.join(products)
        # print(f'index: {index}, query: {row["query"]}\nproducts: {products}\nnew_query: {train_df.at[index,"query_with_products"]}')



  2%|▏         | 124/5460 [00:28<20:23,  4.36it/s]


KeyboardInterrupt: 

C. Get Prompt for each type

In [26]:
train_df['cot_prompt_by_category'] = train_df.apply(lambda row: prompt_map[row['type_']].replace('<QUERY>',row['query_with_products']),axis=1)

### 2.Label

In [27]:
train_df['cot_label_by_category'] = '<output>\n' + train_df['cot_label'] + '\n</output>'
# 格式与Prompt示例统一化
train_df.cot_label_by_category = train_df.cot_label_by_category.str.replace('\n','\n    ')
train_df.cot_label_by_category = train_df.cot_label_by_category.str.replace('\t\t\t','    ')
train_df.cot_label_by_category = train_df.cot_label_by_category.str.replace('\t\t','')
train_df.cot_label_by_category = train_df.cot_label_by_category.str.replace('    </output>','</output>')

In [28]:
train_df[['query','cot_prompt_by_category','cot_label_by_category']].to_excel('../data/data4cot/cot4train_by_category.xlsx',index=False)

### 3. 抽样观察

In [ ]:
n = random.choice(train_df.index)
print(f'第{n}条数据')
print(train_df.iloc[n,1])
print('========================================================================================================')
print(train_df.iloc[n,2])